In [ ]:
#spark.sql('show databases').show()
import pandas as pd
pd.set_option('display.max_columns', 100)
spark.sql('use covid_2020_q3')

In [ ]:
spark.sql('show tables').toPandas()

In [ ]:
### Diagnosis
spark.sql('create table LR_dec.LR_cond as select distinct personid,encounterid, (case when effectivedate=="" then asserteddate else effectivedate end) diagdate,codetype, conditioncode, source_encounter_type from condition ' )
spark.sql('create table LR_dec.LR_cond2 as select distinct i.*,c.servicedate ,c.hospitalizationstartdate ,c.dischargedate from LR_dec.LR_cond i left join encounter c  on i.personid=c.personid and i.encounterid=c.encounterid')
spark.sql('create table LR_dec.lr_all_diag1 as select distinct c.personid, codetype, conditioncode,(case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end) vdate ,(case when diagdate="" then (case when dischargedate="" then servicedate else dischargedate end) else diagdate end) event_date from LR_dec.LR_cond2 c, LR_dec.LR_cohort_outcome_demo i where c.personid=i.personid and '
          + ' datediff((case when diagdate="" then (case when dischargedate="" then servicedate else dischargedate end) else diagdate end) , i.max_1rstenc_dis_dt)<1 ')
### Create data for Inhospital mortality diag
#spark.sql('drop table LR_Dec.LR_inhosp_mort_diag_tmp')
spark.sql('create table LR_Dec.LR_inhosp_mort_diag_tmp as  select distinct d.personid, concat("D_",(case when codetype like ("%ICD%10%") then "ICD10" when codetype like ("%ICD%9%D%") then "ICD9" when codetype like ("%SNOMED%") then "SNOMED" end),"_",conditioncode) covar, to_date(vdate) enc_date from LR_Dec.lr_all_diag1 d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and (codetype like ("%ICD%10%") or codetype like ("%ICD%9%D%") or codetype like ("%SNOMED%")) and datediff(d.event_date,p.min_adm_date)<1')

In [ ]:
### Medication
## extract the basic data
spark.sql('create table LR_DEC.LR_med_all as select distinct personid,encounterid,(case when startdate="" then (case when stopdate="" then NULL else stopdate end) else startdate end) med_date,codetype,drugcode,drug,dosequantity,doseunit,route,frequency,"med_table" d_source  from medication where lower(nvl(status,"Incomplete"))in ("complete","active","discontinued","suspended","incomplete","drug compliance good","administer","started","increased") and personid in (select distinct personid from LR_dec.LR_cohort_outcome_demo) '
+' union select distinct personid,encounterid,(case when startdate="" then (case when stopdate="" then NULL else stopdate end) else startdate end) med_date,codetype,drugcode,drug,dosequantity,doseunit,route,frequency,"med_rec_table" d_source  from med_rec_compliance where lower(nvl(action,"Incomplete"))not in ("error entry deleted","cancelled","canceled") and lower(nvl(compliance,"Incomplete"))not in ("does not take medication") and personid in (select distinct personid from LR_dec.LR_cohort_outcome_demo)' ).toPandas()
#spark.sql('select * from lr_Dec.LR_med_all where med_date is null' ).toPandas() ## not nice 41234 rows
spark.sql('create table lr_Dec.LR_med_all2 as select distinct i.*,(case when c.dischargedate="" then (case when c.hospitalizationstartdate="" then c.servicedate else  c.hospitalizationstartdate end) else  c.dischargedate end) vdate  from lr_Dec.LR_med_all i left join encounter c  on i.personid=c.personid and i.encounterid=c.encounterid ').toPandas()
#spark.sql('select * from lr_Dec.LR_med_all2 where vdate is null limit 10').toPandas() ###0 perfect
## create mapping used
spark.sql('create table lr_dec.med_map_f1 as select * from (select distinct codetype,drugcode from medication union select distinct codetype,drugcode from med_rec_compliance)a').toPandas()
spark.sql('create table lr_dec.med_map_f2 as select * , (case when codetype like "%MMDC%" then "MMDC" when codetype like "%NDC%" then "NDC" when codetype like "%dNUM%" then "dNUM" when codetype like "%drug%synonym%" then "MDS" when codetype like "%RxNorm%" then "RxNorm" else "other" end) code_type_ab from lr_dec.med_map_f1').toPandas()
spark.sql('create table lr_dec.med_map_p1 as select *, lower(trim(drug_id)) drugid from lr_dec.med_map_f2 a left join multum_dnum_map b on lower(trim(drugcode)) = lower(trim(drug_code))')
spark.sql('create table lr_dec.med_map_p2 as select distinct codetype,drugcode,code_type_ab,code_type ,drug_code,drug_id,drugid from lr_dec.med_map_p1 where code_type_ab=(case when code_type like "%MMDC%" then "MMDC" when code_type like "%NDC%" then "NDC" when code_type like "%dNUM%" then "dNUM" when code_type like "%drug%synonym%" then "MDS" when code_type like "%RxNorm%" then "RxNorm" else "other" end)').toPandas()
spark.sql('create table lr_dec.med_map_v1 as select distinct codetype,drugcode,code_type_ab,code_type ,drug_code,drug_id,drugid from lr_dec.med_map_p2 where drug_id is not null').toPandas()
## Medication and categories in Multum
spark.sql('create table lr_Dec.LR_med_all3 as select distinct personid, "MultumdNUM" medcodetype, lower(trim(drugcode)) med_code, vdate , med_date from lr_Dec.LR_med_all2 m where codetype like "%dNUM%"  '
        + ' union select distinct personid, "MultumdNUM" medcodetype, d.drugid med_code, vdate , med_date from lr_dec.LR_med_all2 m , lr_dec.med_map_v1 d where m.codetype=d.codetype and m.drugcode=d.drugcode'
        + ' union select distinct personid, concat("unmap_",m.codetype) medcodetype, m.drugcode med_code, vdate , med_date from lr_dec.LR_med_all2 m left join lr_dec.med_map_v1 d on m.codetype=d.codetype and m.drugcode=d.drugcode where drug_id is null')
spark.sql('create table lr_dec.LR_inhosp_mort_med_tmp as  select distinct d.personid, concat("M_",medcodetype,"_",med_code) covar, to_date(vdate) enc_date from lr_dec.LR_med_all3 d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and datediff(nvl(d.med_date,d.vdate),p.min_adm_date)<1').toPandas()
spark.sql('create table lr_dec.LR_inhosp_mort_med_cat_tmp as  select distinct d.personid, concat("M_",medcodetype,"_",cast(multum_category_id as string)) covar, to_date(vdate) enc_date from (select distinct personid,"MultumCat" medcodetype,multum_category_id, vdate ,med_date from lr_dec.LR_med_all3 ,multum_drug_category_map where drug_id=med_code and medcodetype not like "unmap%") d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and datediff(nvl(d.med_date,d.vdate),p.min_adm_date)<1').toPandas()

In [ ]:
#### Create lab information
spark.sql( 'create table lr_Dec.LR_all_results_pre1 as '
+ 'select distinct r.personid , r.encounterid , codetype , resultcode , result,resulttype, to_date(r.servicedate) result_date, ' 
+ ' nvl((case when interpretation="" then NULL else interpretation end ),'
+ ' nvl((case when codifiedvalue="" then NULL else codifiedvalue end) ,'
+ ' nvl((case when (codifiedvaluecode="" or codifiedvaluecode is NULL )then NULL else (concat(codifiedvaluecodetype,"_",codifiedvaluecode)) end),'
+ ' nvl((case when textvalue="" then NULL else textvalue end),'
+ ' (case when numericvalue is not null then concat(cast(numericvalue as string),unitofmeasure) else "NoResult" end)))))result_summary '
+ ' from result r, lr_Dec.LR_cohort_outcome_demo i '
+ ' where r.personid is not null and i.personid=r.personid and to_date(r.servicedate) <= to_date(i.max_1rstenc_dis_dt)')
spark.sql('create table lr_dec.LR_all_results_pre2 as select i.*,(case when c.dischargedate="" then (case when c.hospitalizationstartdate="" then c.servicedate else  c.hospitalizationstartdate end) else  c.dischargedate end) vdate  from lr_dec.LR_all_results_pre1 i left join encounter c  on i.personid=c.personid and i.encounterid=c.encounterid')
spark.sql('create table lr_dec.LR_all_results_pre3 as select distinct personid, codetype,concat(resultcode,"$",result_summary) value, vdate, result_date from lr_dec.LR_all_results_pre2 where result_summary>="A" and lower(result_summary) not in ("not applicable","unknown")') ### perfect no null or empty colmns
spark.sql('create table lr_dec.LR_inhosp_mort_lab_tmp as  select distinct d.personid, concat("L_",codetype,"_",value) covar, to_date(vdate) enc_date from lr_dec.LR_all_results_pre3 d, lr_dec.LR_inhosp_mort_tmp p where d.personid=p.personid and datediff(d.result_date,to_date(p.min_adm_date))<1').toPandas()

spark.sql('create table lr_dec.LR_inhosp_mort_lab_tmp_v1 as select *, split(covar,"_")[1] codetype,split(replace((split(covar,"_")[2]),"$","#"),"#")[0] test_code ,split(replace((split(covar,"_")[2]),"$","#"),"#")[1] result  from  lr_dec.LR_inhosp_mort_lab_tmp ').toPandas()
spark.sql('create table lr_dec.LR_inhosp_mort_lab_tmp_v2 as select *, split(covar,"_")[1] codetype,split(replace((split(covar,"_")[2]),"$","#"),"#")[0] test_code ,split(replace((split(covar,"_")[2]),"$","#"),"#")[1] result,split(covar,"_")[3] codified_value  from  lr_dec.LR_inhosp_mort_lab_tmp ').toPandas()
spark.sql('create table lr_dec.LR_inhosp_mort_lab_tmp_v3 as select *, concat("L_",(case when upper(codetype) like "SNOMED%" then "SNOMED" else upper(codetype) end),"_",upper(test_code), '
          +                     ' (case when ( result="" or result is null or lower(result) like "%noresult%" or lower(result) like "result%" or lower(result) like "%comment%" or lower(result) like "%note%" or lower(result) like "see%" ) then "" ' 
          +                     '  when ((codified_value is not null and codified_value!="") and (result in ("NCI","LOINC")) ) then concat("$",upper(trim(result)),"|",upper(trim(codified_value)))  '
          +                     '  when ((codified_value is not null and codified_value!="") and (result in ("SNOMED CT")) ) then concat("$","SNOMED","|",upper(trim(codified_value)))  '
          +                     '  when ((codified_value is not null) and (result not in ("NCI","LOINC","SNOMED CT")) ) then "" else concat("$",lower(trim(result))) end )) covar_updated '
          +                     ' from lr_dec.LR_inhosp_mort_lab_tmp_v2 where codetype like "L%" or codetype like "S%" or codetype like "N%" ' ).toPandas()
spark.sql('create table lr_dec.LR_inhosp_mort_lab_tmp_v4 as select * from (select distinct personid, concat("L_",(case when upper(codetype) like "SNOMED%" then "SNOMED" else upper(codetype) end),"_",upper(test_code)) covar , enc_date from lr_dec.LR_inhosp_mort_lab_tmp_v3 '
          +                     ' union select distinct personid, covar_updated covar , enc_date from lr_dec.LR_inhosp_mort_lab_tmp_v3) x ' ).toPandas()

In [ ]:
##### Proedures

spark.sql('create table LR_dec.LR_proc as select distinct personid,encounterid, (case when servicestartdate=="" then serviceenddate else servicestartdate end) procdate,codetype, procedurecode from procedure ' )
spark.sql('create table LR_dec.LR_proc2 as select distinct i.*,c.servicedate ,c.hospitalizationstartdate ,c.dischargedate from LR_dec.LR_proc i left join encounter c  on i.personid=c.personid and i.encounterid=c.encounterid')
spark.sql('create table LR_dec.lr_all_proc1 as select distinct c.personid, codetype, procedurecode,(case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end) vdate ,(case when procdate="" then (case when dischargedate="" then servicedate else dischargedate end) else procdate end) event_date from LR_dec.LR_proc2 c, LR_dec.LR_cohort_outcome_demo i where c.personid=i.personid and '
          + ' datediff((case when procdate="" then (case when dischargedate="" then servicedate else dischargedate end) else procdate end) , i.max_1rstenc_dis_dt)<1 ')

spark.sql('create table LR_Dec.LR_inhosp_mort_proc_tmp as  select distinct d.personid, concat("P_",(case when codetype like ("%CPT-4%") then "CPT4" when codetype like ("%ICD%9%P%") then "ICD9PCS" when codetype like ("%ICD%10%P%") then "ICD10PCS" when codetype like ("HCPCS%") then "HCPCS" when codetype like ("%SNOMED%") then "SNOMED" else upper(trim(codetype)) end),"_",upper(trim(procedurecode))) covar, to_date(vdate) enc_date from LR_Dec.lr_all_proc1 d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and (codetype in ("ICD-10-PCS","HCPCS","ICD-9-CM Procedures","HCPCS Level II","CPT-4 (HCPCS Level I)","LOINC","SNOMED CT")) and datediff(d.event_date,p.min_adm_date)<1')



In [ ]:
### the plan for demographics
# use table lr_dec.LR_inhosp_pt_dts_encs select pt, a_ max age, enc_date, group by personid , enc_date
spark.sql('create table lr_dec.LR_inhosp_pt_dts_encs as select distinct a.*,b.encounterid,b.age_at_encounter from lr_dec.LR_inhosp_pt_dts a left join encounter b on a.personid=b.personid and a.enc_date= to_date((case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end)) ').toPandas()
# gender male/female other_unknown 
# race 1. White /Caucasian  2. Black or African American 3.Asian or Pacific Islander 4.American Indian or Alaska Native
# 5. other_unknown  ethinicty hispanic
# Region 1. Northeast 2. Midwest 3. South 4. West
## Devision  1. New England 2. Middle Atlantic 3. East North Central  4. West North Central 5. South Atlantic 6. East South Central 7. West South Central 8. Mountain 9. Pacific

spark.sql('create table lr_dec.LR_inhosp_demo_tmp as select distinct p.personid, concat("g_",( case when lower(trim(d.gender)) in ("female","male") then lower(trim(d.gender)) else "other"  end))covar,enc_date from demographics_corrected d, lr_dec.LR_inhosp_pt_dts p where d.personid=p.personid '
         +' union select personid, concat("a_",cast(max(age_at_encounter) as string))covar,enc_date from lr_dec.LR_inhosp_pt_dts_encs group by personid,enc_date '
         +' union select distinct p.personid, concat("r_",( case when lower(race) like "%white%" then "1"  when lower(race) like "%african%american%" then "2" when lower(race) like "%asian%" then "3" when lower(race) like "%alaska%native%" then "4" else "5"  end))covar,enc_date from demographics_corrected d, lr_dec.LR_inhosp_pt_dts p where d.personid=p.personid '
         +' union select distinct p.personid, "e_hispanic" covar,enc_date from demographics_corrected d, lr_dec.LR_inhosp_pt_dts p where d.personid=p.personid and trim(ethnicity)="Hispanic or Latino"'
         +' union select distinct p.personid, concat("cr_",substring(trim(censusRegion),1,1))covar,enc_date from demographics_corrected d,lr_dec.LR_inhosp_pt_dts p , tenant t where d.personid=p.personid and d.tenant=t.tenantid'
         +' union select distinct p.personid, concat("cd_",substring(trim(censusDivision),1,1))covar,enc_date from demographics_corrected d,lr_dec.LR_inhosp_pt_dts p, tenant t where d.personid=p.personid and d.tenant=t.tenantid'
         ).toPandas()


In [ ]:
##### Cleaning up the patient information to create the more clean codes ( to avoid duplicate SNOMED and Loincs)
spark.sql('create table lr_Dec.LR_inhosp_pred_diag_v1 as select distinct personid ,substring(replace(covar,".",""), 3) new_covar ,enc_date from lr_Dec.LR_inhosp_mort_diag_tmp' ).toPandas()
spark.sql('create table lr_Dec.LR_inhosp_proc_proc_v1 as select distinct personid ,substring(replace(covar,".",""), 3) new_covar ,enc_date from LR_Dec.LR_inhosp_mort_proc_tmp').toPandas()
spark.sql('create table lr_Dec.LR_inhosp_pred_result_v1 as select distinct personid ,substring(covar, 3) new_covar ,enc_date  from lr_dec.LR_inhosp_mort_lab_tmp_v4 ').toPandas()
spark.sql('create table lr_Dec.LR_inhosp_pred_med_v1 as select distinct personid ,substring(covar, 3) new_covar ,enc_date from (select * from lr_dec.LR_inhosp_mort_med_tmp where covar not like "%M_u%" union select * from lr_dec.LR_inhosp_mort_med_cat_tmp) x ').toPandas()
spark.sql('select distinct personid ,covar new_covar ,enc_date from LR_Dec.LR_inhosp_demo_tmp ').toPandas()

# Dump data
spark.sql('select distinct x.* from (select * from lr_Dec.LR_inhosp_pred_diag_v1 union select * from  lr_Dec.LR_inhosp_proc_proc_v1 union select * from lr_Dec.LR_inhosp_pred_result_v1 union select * from lr_Dec.LR_inhosp_pred_med_v1 union select distinct personid ,covar new_covar ,enc_date from LR_Dec.LR_inhosp_demo_tmp ) x order by personid, enc_date desc').write.option("header","true").csv('LR_inhospDec_dmlpd_all_dat.csv')
spark.sql('select distinct personid, mort_enc1_flag,enc1_los,vent_flag, nvl(datediff(vent_date,min_adm_date),enc1_los) tti,readm_flag,(case when enc1_los>7 then "1" else "0" end) plos_flag from lr_dec.LR_cohort_outcome_demo where personid in (select distinct personid from lr_dec.LR_inhosp_pt_dts ) and mort_enc1_flag= Mortality_flag and enc1_los>=1 and to_date(nvl(vent_date,"1900-01-01T16:12:00+00:00")) <= to_date(max_1rstenc_dis_dt) ').write.option("header","true").csv('LR_inhospDec_dmlpd_all_labelv1.csv')


In [ ]:
### Cleaning up the database
#spark.sql('drop table lr_dec.med_map_f1')
#spark.sql('drop table lr_dec.med_map_f2')
#spark.sql('drop table lr_dec.med_map_p1')
#spark.sql('drop table lr_dec.med_map_p2')
#spark.sql( 'drop table lr_Dec.LR_all_results_pre1')
#spark.sql( 'drop table lr_Dec.LR_all_results_pre2')